In [1]:
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
import pandas as pd
import numpy as np
import torch

In [2]:
wiki = load_dataset('wiki_snippets', 'wiki40b_en_100_0', split='train')
eli5 = load_dataset('eli5', split='train_eli5')

In [3]:
random_question_id = np.random.randint(0, len(eli5))
random_question_id

247899

In [4]:
import psycopg2

conn = psycopg2.connect(database="eli5",
                        host="127.0.0.1",
                        user="admin",
                        password="1234",
                        port="6000")

cursor = conn.cursor()

query = 'SELECT * FROM wiki40b;'

cursor.execute(query)
data =cursor.fetchall()

In [5]:
sbert_model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [6]:
def sort_key(item):
    return item[2]

In [7]:
question = eli5[random_question_id]['title']
q_embed = torch.Tensor(sbert_model.encode(question))

In [8]:
for item_idx in range(len(data)):
    ref_emb = data[item_idx][1]
    ref = ref_emb[1:-1].split(',')
    ref = torch.Tensor([float(x) for x in ref])
    item_similarity = util.cos_sim(q_embed, ref)
    data[item_idx] = data[item_idx] + (item_similarity,)
data.sort(key = lambda row: row[2], reverse=True)

In [9]:
top_5 = data[:5]

In [10]:
df = pd.DataFrame({
    question: [wiki[x]['passage_text'] for x in [x[0] for x in top_5]] 
}
)

In [11]:
df.style.set_properties(**{'text-align': 'left'})

,How do ancient cities get buried and completely underground with time?
0,"introduce new theories about how ancient civilizations lived and died, and to uncover the origin of each mummy."
1,"In the examples built before the l6th century the dome is covered with a conical or pyramidal spire. Most of these monuments are two storied. Concealed inside the base, half of which is below ground level, is a crypt; the latter is covered by a vault and its floor is earth. The deceased was buried in the earth. The crypt had small loop-hole windows. Description Kümbet Yaylası (""Kuembet Plateau"") is reached via Giresun-Dereli-Şebinkarahisar route which is 60 km from Giresun city centrum. The Plateau of Kümbet is 1640 meters high and its southern part is in Alpine zone. The plateau has"
2,"castle was provided with a girding wall and a well shaft. Nevertheless, the castle was forsaken before the Middle Ages had even ended, and hence, a written mention of it is nowhere to be found. In the 19th century, the knoll became the location for something else, a hard-stone quarry. The work there removed any remnant of the old castle that had lain there for centuries. Also unearthed at the site were pieces of Roman grave monuments, which might themselves have been used later to build the castle. In bygone centuries, the municipal area was a tangle of crisscrossing property"
3,"what is now the main cave is relatively recent, and that some 350,000 years ago water sinking at Rumbling Hole followed a phreatic trunk route along the fault, entering Death's Head Hole at the end of East Passage, flowed across what is now the Main Chamber, into the passages below Big Meanie. From here the water flowed through a continuation of the now-blocked passage to Glasfurds Chamber in Gavel Pot, and hence to a resurgence in the Leck Beck Valley some 100 metres (330 ft) above the current resurgence, which is now covered with glacial till. History Balderstone described the entrance"
4,"as a Grand Duke of Baldur's Gate would be held, and present evidence of his schemes. Exposed, Sarevok flees into an ancient underground ruin beneath Baldur's Gate, with the Ward and the party following after. The Ward confronts Sarevok within an ancient temple to Bhaal, and defeats him, saving the Sword Coast and ending their brother's schemes. In the final ending cinematic, Sarevok's tainted soul departs his body and travels deep underground to a large circular chamber of alcoves, and destroys a statue of himself contained in one of the alcoves, whereupon it is revealed that the other alcoves each"
